In [36]:
import os, ast, glob
import pandas as pd

In [32]:
def calculate_true_positive(predicted_list, correct_list):
    tp = 0
    for predicted_compound_target in predicted_list:
        if predicted_compound_target in correct_list:   # 'screen' <- predicted 'screen'
            correct_list.remove(predicted_compound_target)
            tp += 1
            continue
        for correct_target in correct_list:
            if predicted_compound_target.find(correct_target) > -1:   # 'audio' <- predicted 'audio aspects'
                correct_list.remove(correct_target)
                tp += 1
                break
    return tp

In [55]:
pattern = 'nsubj'
dfs = []
for pattern in ['nsubj', 'amod', 'obl']:
    filepaths = glob.glob(os.path.join(os.path.dirname(os.getcwd()), 'output', 'preliminary', 'targets', 'Wireless router_*_%s.csv'%pattern))
    df = pd.read_csv(filepaths[0])
    df['targets'] = df.apply(lambda x: ast.literal_eval(x['targets']), axis=1)
    df['predicted_targets'] = df.apply(lambda x: ast.literal_eval(x['predicted_targets']), axis=1)
    df['tp'] = df.apply(lambda row: calculate_true_positive(row['predicted_targets'], row['targets']), axis=1)
    pattern_features_df = df[['content', 'tp']]
    pattern_features_df['pattern'] = pattern
    dfs.append(pattern_features_df)
concat_df = pd.concat(dfs, ignore_index=True)    

/home/dmlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [51]:
x1 = x1.reshape(-1, 1)
x1.shape

(40, 1)

In [62]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif

# le = preprocessing.LabelEncoder()
# le.fit(concat_df['content'])
# print(list(le.classes_))

# print(le.transform(current_df['content']))
# print(list(le.inverse_transform([2, 2, 1])))

# x1 = le.transform(concat_df[concat_df['pattern']=='nsubj']['content']).reshape(-1, 1)
# x2 = le.transform(concat_df[concat_df['pattern']=='amod']['content']).reshape(-1, 1)

pattern1, pattern2 = 'nsubj', 'amod'
x1 = concat_df[concat_df['pattern']==pattern1]['tp'].values.reshape(-1,1)
x2 = concat_df[concat_df['pattern']==pattern2]['tp'].values.reshape(-1,1)
print('%s & %s => %.4f' % (pattern1, pattern2, mutual_info_classif(x1, x2)[0]))

pattern1, pattern2 = 'nsubj', 'obl'
x1 = concat_df[concat_df['pattern']==pattern1]['tp'].values.reshape(-1,1)
x2 = concat_df[concat_df['pattern']==pattern2]['tp'].values.reshape(-1,1)
print('%s & %s => %.4f' % (pattern1, pattern2, mutual_info_classif(x1, x2)[0]))

nsubj & amod => 0.0019
nsubj & obl => 0.0022


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [63]:
pattern1, pattern2 = 'amod', 'obl'
x1 = concat_df[concat_df['pattern']==pattern1]['tp'].values.reshape(-1,1)
x2 = concat_df[concat_df['pattern']==pattern2]['tp'].values.reshape(-1,1)
print('%s & %s => %.4f' % (pattern1, pattern2, mutual_info_classif(x1, x2)[0]))

amod & obl => 0.0063


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
